<a href="https://colab.research.google.com/github/aswinaus/RAG/blob/main/Copy_of_rag_pipeline_pymupdf_multiple_docs_persisted_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
!pip install pdfminer.six langsmith langchain langchain-community langchain_openai chromadb pypdf nest_asyncio --quiet

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [ ]:
# Download Data
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

In [ ]:
# step 1: upload a PDF to the root of the file browser
# then change the variable below to be the name of your file
file_name = 'lyft_10k_2023'

In [ ]:
pdf_text = extract_text_from_pdf(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf")

In [ ]:
pdf_text.split('\n')[0:10000]

In [ ]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
loader = PyPDFLoader(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf")
#loader = PyPDFLoader(f"{data_dir}/RAG/data/10k/uber_10k_2023.pdf")
# load_and_split uses RecursiveCharacterTextSplitter by default
pages = loader.load_and_split(text_splitter)

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# create vector store with Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata # import filter_complex_metadata
vectordb = Chroma.from_documents(documents=pages, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]), persist_directory=f"{data_dir}/RAG/VectorDB/chroma_db")
vectordb.persist()

In [ ]:
# trying with multi-query
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model Accounting assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
question = "What would be the best possible way to evaluate the Intangibles in Transfer Pricing?"

In [ ]:
generate_queries.invoke(question)

In [ ]:
# Retrieve docs given a list of queries
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
# Rank documents
from langchain.load import dumps, loads

def rank_documents(results: list[list], k=60):
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return reranked_results

retrieval_chain = generate_queries | retriever.map() | rank_documents
docs = retrieval_chain.invoke(question)

In [ ]:
#to check the source and context
docs

In [ ]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

# Chain
final_rag_chain = (
    {"context": retrieval_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

In [ ]:
questions = [
    "Please explain the revolving credit of Lyft for year 2023 and its long term benefits?",
    "Could you give me a concise overview of the Business strategies employed by Uber and Lyft?",
    "What are the details of Lyft's revolving credit for the year 2023 and how does it contribute to long-term benefits?",
    "Could you analyze and compare the financial health of Uber and Lyft for the year 2023?",
    "When compared to Uber is Lyft financially stable?",
    "Can you compare the growth strategies between Uber and Lyft based on the 10K document for year 2023?",
    "Can you provide a summary of the Financial and Operational Highlights of Uber?"
    "How would you describe the Business overview of Uber and Lyft in a nutshell?",
    "How does Lyft's financial stability in 2023 compare to Uber's?",
    "Can you provide an overview of Lyft's revolving credit in 2023 and its potential long-term advantages?",
    "Can you please compare the financial status for both the companies for the year 2023?",
    "Can you compare the cost and expenses between Lyft and Uber for year 2023? Also let me know which company faired better?",
    "Can you provide a financial comparison between Uber and Lyft for the year 2023?",
    "Could you give me an overview of Lyft's Financial and Operational performance?",
    "In terms of financial stability, how does Lyft stack up against Uber for the year 2023?",
    "How does Lyft's revolving credit in 2023 work and what are the advantages it offers in the long run?",
    "When it comes to technology which one fairs better is it Uber or Lyft?",
    "What are the contrasting growth approaches of Uber and Lyft as outlined in the 2023 10K report?",
    "Could you give me a concise overview of the Business strategies employed by Uber and Lyft?",
    "Could you analyze and compare the financial health of Uber and Lyft for the year 2023?",
    "Can you summarize the important Financial and Operational metrics for Lyft",
    "When evaluating financial stability, how do Uber and Lyft differ in their performance for the year 2023?"
]
#context=["Revolving Credit Facility & Other Financings On November 3, 2022, Lyft, Inc. entered into a revolving credit agreement (the “Existing Revolving Credit Agreement”) by and among the Company, as the borrower, JPMorgan Chase Bank, N.A., as administrative agent, and certain lenders party thereto from time to time. On December 12, 2023, the Company entered into Amendment No. 1 to Revolving Credit Agreement (the “Credit Agreement Amendment”) with the other loan parties party thereto,JPMorgan Chase Bank, N.A., as administrative agent and certain lenders party thereto, which amends the Existing Revolving Credit Agreement (the Existing Revolving Credit Agreement as amended by the Credit Agreement Amendment, the “Revolving Credit Agreement”) to, among other things, (i) permit the Company to refinance existing junior indebtedness (including the 2025 Notes) with proceeds from one or more new convertible debt issuance(s) or other subordinated indebtedness, subject to certain conditions set forth therein, (ii) permit the Company to repurchase up to $450.0 million of the 2025 Notes, (iii) extend the applicability of the existing liquidity covenant in the Revolving Credit Agreement to the fiscal quarter ending June 30, 2024 and (iv) commence the date of the stepdown of the total leverage ratio in the Revolving Credit Agreement from 3.50x to 3.00x at the fiscal quarter ending March 31, 2025.",
#]
ground_truth = [
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Lyft, Inc. started a movement to revolutionize transportation in 2012 with a peer-to-peer marketplace for on-demand ridesharing. They have continued to pioneer innovations and are now one of the largest multimodal transportation networks in the United States and Canada. Lyft's purpose is to get riders out into the world and provide drivers with control over their time and money. They offer a ridesharing marketplace through the Lyft App, connecting drivers with riders and providing various transportation modes. Uber Technologies, Inc. is a technology platform that powers movement from point A to point B. They connect consumers with providers of ride services, merchants, and delivery service providers. Uber also connects consumers with public transportation networks and carriers in the freight industry. They are developing technologies to provide new solutions for everyday problems.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft reported total costs and expenses of 36,171 million. Comparing the two, Lyft had higher costs and expenses than Uber for the year 2023. Therefore, Uber fared better in terms of managing costs and expenses in 2023."
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. G ross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "In 2023, Lyft entered into a revolving credit agreement with certain lenders for a 1.25 billion.  The long-term benefits of this revolving credit facility include providing Lyft with financial flexibility and access to capital when needed. It allows Lyft to borrow funds, repay them, and borrow again up to the agreed-upon limit, providing liquidity for operational needs, investments, or other strategic initiatives. Additionally, having a revolving credit facility in place can help Lyft manage its cash flow, take advantage of opportunities for growth, and navigate any unforeseen financial challenges.",
    "Based on the provided context, both Uber and Lyft heavily rely on technology to power their transportation services. Uber is described as a technology platform that uses leading technology to connect consumers with ride services, delivery services, and public transportation networks. Lyft, on the other hand, leverages its technology platform to connect drivers with riders through its ridesharing marketplace and offers an expanded set of transportation modes. In terms of technology, both Uber and Lyft have their strengths and focus areas. Uber emphasizes its technology applications supporting a variety of offerings on its platform, while Lyft highlights its robust technology platform that powers rides and connections every day. Ultimately, the effectiveness and success of each company's technology may vary based on specific features, innovations, and user experiences.",
    "Based on the information provided in the 10K documents for 2023, both Uber and Lyft have growth strategies focused on increasing their user base, expanding their range of transportation services, and capturing a larger share of the transportation market. Lyft's growth strategy includes increasing rider use cases by offering various products and services such as Lyft Pink subscription plan, Lyft Pass commuter programs, and first-mile and last-mile services. They also aim to grow their share of consumers' transportation spend by providing a wide range of mobility options through their rideshare, bikes, and scooters network.On the other hand, Uber's growth strategy involves using their technology platform to connect consumers with various ride services, merchants, and delivery service providers. They also connect consumers with public transportation networks and provide solutions in the freight industry. Uber is also developing new technologies to solve everyday problems.Both companies face competition from other players in the market, and they are focused on innovation, technology, and expanding their services to stay competitive and attract more users.",
    "Lyft, Inc. started a movement to revolutionize transportation in 2012 with a peer-to-peer marketplace for on-demand ridesharing. They have continued to pioneer innovations and are now one of the largest multimodal transportation networks in the United States and Canada. Lyft's purpose is to get riders out into the world and provide drivers with control over their time and money. They offer a ridesharing marketplace through the Lyft App, connecting drivers with riders and providing various transportation modes.Uber Technologies, Inc. is a technology platform that powers movement from point A to point B. They connect consumers with providers of ride services, merchants, and delivery service providers. Uber also connects consumers with public transportation networks and carriers in the freight industry. They are developing technologies to provide new solutions for everyday problems.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023.",
    "Financial Highlights: Lyft, Inc. operates multimodal transportation networks in the United States and Canada. The company's revenue is primarily generated from its ridesharing marketplace connecting drivers and riders. Lyft collects service fees and commissions from drivers for their use of the ridesharing marketplace. Gross Bookings and Rides increase as drivers accept more rider leads.Operational Highlights: Lyft offers an expanded set of transportation modes in select cities, including shared bikes and scooters for shorter rides and multimodal trips. The company's platform and mobile-based applications facilitate peer-to-peer ridesharing by connecting drivers with riders. Lyft aims to grow its share of consumers' transportation spend by providing a wide range of mobility options. The company focuses on delivering increasing value to drivers by offering economic opportunities and programs like Express Drive for access to rental cars.",
    "Based on the provided context, it appears that Lyft is in a better financial position compared to Uber for the year 2023. Lyft reported revenue of 37,281 million. Additionally, Lyft's net loss percentage was 33.1% compared to Uber's net income percentage of 5%. This indicates that Lyft had a lower loss percentage compared to Uber's income percentage, suggesting that Lyft may be more financially stable in 2023."
]

In [ ]:
!pip install datasets --quiet
from datasets import Dataset

In [ ]:
answers  = []
contexts = []

# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(final_rag_chain.invoke({"question":question}))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

dataset.to_pandas()

In [ ]:
!pip install ragas --quiet
import ragas

In [ ]:
#Evaluate using RAGAS
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()
df